In [1]:
%matplotlib inline
import matplotlib
from matplotlib import pyplot as plt

import qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, IBMQ
from qiskit import transpile, schedule, assemble

In [2]:
# loading account
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')

# get simulator f_alma:
from qiskit.test.mock import FakeAlmaden
f_alma = FakeAlmaden() # simulator f_alma for pulse schedules

# All including function

In [4]:
# func to apply protocol for 7 qubits, expected fidelity of teleportation: 50%

def apply_7_prot(circ, meas_pair_num):
    
    '''
    args: 
        circ (QuantumCircuit object) circuit to apply protocol to
        meas_pair_num (int) index num of EPR pair used for basis measurement
    '''
    
    bell_pairs = [(5,6),(2,3),(1,4)]
    qubits_to_measure = bell_pairs[meas_pair_num]
    
    # Prepare state of alice q0 to be |1> 
    circ.x(0)
    
    # Define many-body-system bell pairs
    for pair in bell_pairs:
        circ.h(pair[0])
        circ.cx(pair[0],pair[1])

    # apply scrambling unitary U
    circ.cz(0,2)
    circ.cz(0,1)
    circ.cz(1,2)
    circ.h(0)
    circ.h(1)
    circ.h(2)
    circ.cz(0,2)
    circ.cz(0,1)
    circ.cz(1,2)

    # apply conjugate of U
    circ.cz(3,5)
    circ.cz(4,5)
    circ.cz(3,4)
    circ.h(3)
    circ.h(4)
    circ.h(5)
    circ.cz(3,5)
    circ.cz(4,5)
    circ.cz(3,4)   

    # apply bell basis measurement
    circ.cx(qubits_to_measure[0], qubits_to_measure[1])
    circ.h(qubits_to_measure[1])
    circ.measure(qubits_to_measure[0], qubits_to_measure[0])
    circ.measure(qubits_to_measure[1], qubits_to_measure[1])
    
    # bob measurement
    circ.measure(6,6) 

## Tests

In [10]:
## TEST SIM 1:

# test meas_pair_num 0
meas_pair_num = 0

# initiate 7 qubit circuit
circ = QuantumCircuit(7,7)

# apply protocol
apply_7_prot(circ, meas_pair_num)

# run on f_alma
pct = 250
shots = 100*pct
qobj = assemble(circ, shots=shots)
result = f_alma.run(qobj).result().get_counts()
bob_meas_1 = sum([v for (k,v) in result.items() if k[0]=="1"])
bob_meas_0 = sum([v for (k,v) in result.items() if k[0]=="0"])
print("\n meas_pair_num_0:\tFidelity {}%".format(bob_meas_1/pct))  


## TEST SIM 2:

# test meas_pair_num 1
meas_pair_num = 1

# initiate 7 qubit circuit
circ = QuantumCircuit(7,7)

# apply protocol
apply_7_prot(circ, meas_pair_num)

# run on f_alma
pct = 250
shots = 100*pct
qobj = assemble(circ, shots=shots)
result = f_alma.run(qobj).result().get_counts()
bob_meas_1 = sum([v for (k,v) in result.items() if k[0]=="1"])
bob_meas_0 = sum([v for (k,v) in result.items() if k[0]=="0"])
print("\n meas_pair_num_1:\tFidelity {}%".format(bob_meas_1/pct)) 


## TEST SIM 3:

# test meas_pair_num 2
meas_pair_num = 2

# initiate 7 qubit circuit
circ = QuantumCircuit(7,7)

# apply protocol
apply_7_prot(circ, meas_pair_num)

# run on f_alma
pct = 25
shots = 100*pct
qobj = assemble(circ, shots=shots)
result = f_alma.run(qobj).result().get_counts()
bob_meas_1 = sum([v for (k,v) in result.items() if k[0]=="1"])
bob_meas_0 = sum([v for (k,v) in result.items() if k[0]=="0"])
print("\n meas_pair_num_2:\tFidelity {}%".format(bob_meas_1/pct)) 


 meas_pair_num_0:	Fidelity 49.796%

 meas_pair_num_1:	Fidelity 49.46%

 meas_pair_num_2:	Fidelity 49.88%


# Alternative separate functions

In [11]:
def entangle_bell_pairs(circ, bell_pairs):
    # Define many-body-system bell pairs
    for pair in bell_pairs:
        circ.h(pair[0])
        circ.cx(pair[0],pair[1])

def measure_bell_basis(circ, qubits_to_measure):
    # apply bell basis measurement
    circ.cx(qubits_to_measure[0], qubits_to_measure[1])
    circ.h(qubits_to_measure[1])
    circ.measure(qubits_to_measure[0], qubits_to_measure[0])
    circ.measure(qubits_to_measure[1], qubits_to_measure[1])

In [12]:
def apply_telpro_7(circ):
    
    # for circ with 7 qubits

    # apply scrambling unitary U
    circ.cz(0,2)
    circ.cz(0,1)
    circ.cz(1,2)
    circ.barrier(0,1,2)
    circ.h(0)
    circ.h(1)
    circ.h(2)
    circ.cz(0,2)
    circ.cz(0,1)
    circ.cz(1,2)

    # apply conjugate of U
    circ.cz(3,5)
    circ.cz(4,5)
    circ.cz(3,4)
    circ.barrier(3,4,5)
    circ.h(3)
    circ.h(4)
    circ.h(5)
    circ.cz(3,5)
    circ.cz(4,5)
    circ.cz(3,4)  

## Tests

In [13]:
# test teleportation for 7 qubits

n = 7
circ = QuantumCircuit(n,n)

# teleportation circ
entangle_bell_pairs(circ,bell_pairs=[[1,4],[2,3],[5,6]])
apply_telpro_7(circ)
measure_bell_basis(circ, qubits_to_measure=[2,3])
measure_bell_basis(circ, qubits_to_measure=[5,6])

# run on f_alma
shots = 10000
qobj = assemble(circ, shots=shots)
result = f_alma.run(qobj).result().get_counts()
bob_meas_1 = sum([v for (k,v) in result.items() if k[0]=="1"])
bob_meas_0 = sum([v for (k,v) in result.items() if k[0]=="0"])
print("Fidelity {}%".format(bob_meas_1/100))  

Fidelity 49.02%


In [14]:
# make telpo circ with one additional qubit

n = 7 + 1
circ = QuantumCircuit(n,n)

# teleportation circ
entangle_bell_pairs(circ,bell_pairs=[[1,4],[2,3],[5,6]])
apply_telpro_7(circ)
measure_bell_basis(circ, qubits_to_measure=[2,3])
measure_bell_basis(circ, qubits_to_measure=[5,6])

# run on f_alma
shots = 10000
qobj = assemble(circ, shots=shots)
result = f_alma.run(qobj).result().get_counts()
bob_meas_1 = sum([v for (k,v) in result.items() if k[0]=="1"])
bob_meas_0 = sum([v for (k,v) in result.items() if k[0]=="0"])
print("Fidelity {}%".format(bob_meas_1/100)) 

Fidelity 0.0%
